In [ ]:
from typing import List, Dict, Any

import pandas as pd

import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.getLogger("httpx").setLevel(logging.WARNING)

job_id='dj-synhub-extraction-lkbi9fy6zepu8rcjuxqhjwkbld52wgt0-ouutovygqc'

df = pd.read_csv(f'../../extractions/{job_id}/relevant_articles_to_classify_2025.csv',encoding='utf-8')
df


,source_name,title,company_codes,modification_date,region_codes,segments,num_segments,an,tile_id,company_names
0,Gulf Oil & Gas,Cleanova expands its product offering with the...,"alvmar,cmbndi,dbvpl,percoi",2025-01-10 07:13:57.825,"eurz,uk,weurz",Enhances Cleanova’s presence in the UK and bro...,4,GUONGA0020250109el170007b,1910620,"Alvarez & Marsal Inc,Aon PLC,Debevoise & Plimp..."
1,Gulf Oil & Gas,Cleanova expands its product offering with the...,"alvmar,cmbndi,dbvpl,percoi",2025-01-10 07:13:57.825,"eurz,uk,weurz","\n\nCleanova, a clean tech, market-leading glo...",4,GUONGA0020250109el170007b,1910621,"Alvarez & Marsal Inc,Aon PLC,Debevoise & Plimp..."
2,Gulf Oil & Gas,Cleanova expands its product offering with the...,"alvmar,cmbndi,dbvpl,percoi",2025-01-10 07:13:57.825,"eurz,uk,weurz","\n\nCleanova’s CEO, Javaid Riaz, commented, “T...",4,GUONGA0020250109el170007b,1910622,"Alvarez & Marsal Inc,Aon PLC,Debevoise & Plimp..."
3,Gulf Oil & Gas,Cleanova expands its product offering with the...,"alvmar,cmbndi,dbvpl,percoi",2025-01-10 07:13:57.825,"eurz,uk,weurz",\n\nJavaid Riaz explained what the transition ...,4,GUONGA0020250109el170007b,1910623,"Alvarez & Marsal Inc,Aon PLC,Debevoise & Plimp..."
4,thesun.co.uk,HIGH STREET GIANT WHSmith ‘in secret talks to ...,"bhchcm,brraic,bsbro,lsexch,smwh",2025-01-26 07:33:12.758,"eurz,uk,weurz",The retail group have been in negotiations wit...,7,THESUK0020250125el1p001jl,1910746,"British Chambers of Commerce,British Retail Co..."
...,...,...,...,...,...,...,...,...,...,...
135280,thetimes.co.uk,"Who could buy Bet365? Bankers salivate, but a ...","btsfgl,cnyc,cvccap,drftkn,gamvc,grdmp,habsaa,p...",2025-05-04 07:19:19.010,"eurz,uk,weurz",\n\nJP Morgan was completely unaware that Coat...,7,TIMEUK0020250503el53000jh,2940030,"bet365 Group Limited,JPMorgan Chase & Co.,CVC ..."
135281,thetimes.co.uk,"Who could buy Bet365? Bankers salivate, but a ...","btsfgl,cnyc,cvccap,drftkn,gamvc,grdmp,habsaa,p...",2025-05-04 07:19:19.010,"eurz,uk,weurz",\n\nBet365 has historically had a sizeable rem...,7,TIMEUK0020250503el53000jh,2940031,"bet365 Group Limited,JPMorgan Chase & Co.,CVC ..."
135282,thetimes.co.uk,"Who could buy Bet365? Bankers salivate, but a ...","btsfgl,cnyc,cvccap,drftkn,gamvc,grdmp,habsaa,p...",2025-05-04 07:19:19.010,"eurz,uk,weurz",\n\nBut a sale risks bringing Coates into conf...,7,TIMEUK0020250503el53000jh,2940032,"bet365 Group Limited,JPMorgan Chase & Co.,CVC ..."
135283,thetimes.co.uk,"Who could buy Bet365? Bankers salivate, but a ...","btsfgl,cnyc,cvccap,drftkn,gamvc,grdmp,habsaa,p...",2025-05-04 07:19:19.010,"eurz,uk,weurz","\n\nAccordingly a sale may be her only option,...",7,TIMEUK0020250503el53000jh,2940033,"bet365 Group Limited,JPMorgan Chase & Co.,CVC ..."


In [ ]:
def df_to_prompt_inputs(df: pd.DataFrame, *, business_request: str) -> List[Dict[str, Any]]:
    df_renamed = df.rename(
        columns={
            "source_name": "news_source",
            "title": "article_title",
            "segments": "article_body",
            "company_names": "companies",
            "company_codes": "company_codes",
            "modification_date": "modification_date",
        }
    )
    df_renamed["business_request"] = business_request
    return (
        df_renamed[  # keep the order you want
            [
                "an",
                "news_source",
                "article_title",
                "article_body",
                "companies",
                "company_codes",
                "modification_date",
                "business_request",
            ]
        ]
        .reset_index()  # adds the original index as a column
        .to_dict(orient="records")
    )

In [1]:
from typing import Literal
from pydantic import BaseModel,Field
from llm_utils.factory import construct_llm, LLMChainFactory

class CarveOutAssessment(BaseModel):
    is_co: bool = Field(description="Is the article about a future corporate carve-out?")
    target_company_code: str = Field(description="Code of the company that may be open to divest/carve-out its subsidiary")
    subsidiary_company_code: str = Field(description="Code of the subsidiary that may be open to divest/carve-out")
    is_relevant: bool = Field(description="Is the article relevant to the conditions of business request?")
    co_stage: Literal["Early", "Late"] = Field(description="Stage of the carve-out opportunity")
    short_reasoning: str = Field(description="Justification for the answers provided; not more than 2 sentences")

business_request = """
### Deals search criteria
* Completed date (last 10 years)
* Geography (Europe)

* Deal technique (Divestment)
* Sector (Financial services)
* Size (TBD)

Note that we are looking for future carve-outs, so please do not include any deals that have already been completed.
"""

template = """
# Role and objective: 
You are an expert financial analyst specializing in identifying potential corporate carve-out opportunities in the financial services sector.
Your task is to analyze news articles to determine whether they indicate a company is considering divesting part of its business in the future.
Focus on European financial services companies across ALL segments including banking, wealth management, insurance, financial data,
payment processing, fund administration, brokers, and other financial services.


# Instructions:
    
CRITICAL EVALUATION CRITERIA:
1. MUST IDENTIFY AS CARVE-OUT PROSPECT:
   - Company signals intention to divest specific business units or segments
   - Company announces strategic refocusing that implies non-core units could be sold
   - New strategic plans mentioning focus on specific segments (implying others might be divested)
   - Language about simplifying corporate structure or streamlining operations
   - References to non-core assets, underperforming units, or subscale operations
   - Management discussing 'evaluating options' for specific business segments
    
2. MUST NOT BE INCORRECTLY CLASSIFIED AS CARVE-OUT:
   - Simple stake sales without business unit separation 
   - Private equity firms exiting investments (not corporate carve-outs)
   - IPO plans without specific business unit divestiture
   - Already completed carve-outs (though mention these in reasoning as potentially interesting)
   - General M&A activity without specific divestiture signals
   - Opinion pieces or columns without factual business announcements
    
3. TIMING RELEVANCE:
   - FOCUS ON FUTURE OPPORTUNITIES: Companies currently considering or likely to consider divestitures
   - DE-PRIORITIZE: Already completed transactions (though note these in reasoning)
    
4. KEY SIGNALS OF POTENTIAL CARVE-OUTS:
   - New CEO appointments or management changes
   - Announcements of strategic reviews or new strategic plans
   - Explicit focus on 'core businesses' or 'key segments'
   - Financial pressure (debt issues, dividend concerns, performance challenges)
   - Regulatory challenges that might prompt divestiture
   - Simplification of corporate structure
   - Discontinuation of operations in certain areas
    
When selecting target_company_code:
- If multiple companies might be considering divestitures, select the primary/most likely one
- For corporate groups where a parent may divest subsidiaries, consider both parent and subsidiary codes
- If a company is refocusing on specific segments, this implies other segments could be divested
- Do not pick the wrong company - target company is the one that may divest a subsidiary, a subsidiary is the one that may be divested
    
Business request: {business_request}


Analyze this article and provide a structured assessment of potential carve-out opportunities:

news source: {news_source}
article body: {article_body}
companies: {companies}
company codes: {company_codes}

Think carefully step-by-step about the article and the companies mentioned, and correctly answer if the article is related to carve-out.
"""


llm = construct_llm(model_name='gpt-4.1-mini', temperature=0.1)
factory = LLMChainFactory(model_name='gpt-4.1-mini', temperature=0.1)
so_chain = factory._runnable_with_structured(template=template, response_schemas=CarveOutAssessment, llm=llm, business_request=business_request)

### Screw them batches

In [ ]:
import asyncio
from pathlib import Path
from datetime import datetime
from typing import Dict, Any, List, Tuple
from aiolimiter import AsyncLimiter          

import pandas as pd
from tqdm.auto import tqdm                        # ← use auto for notebook & CLI

# ───────────────────────── 0  Global limiters ──────────────────────────
REQUESTS_PER_MIN = 1_000            # Azure allocation, but burst is 1–2 %
TOKENS_PER_MIN   = 1_000_000

req_limiter = AsyncLimiter(REQUESTS_PER_MIN, 60)   # smooth 60-s bucket
tok_limiter = AsyncLimiter(TOKENS_PER_MIN, 60)

# initial guess for Azure’s 10-s burst bucket (will self-correct)
BURST_BUCKET = 20
burst_sem    = asyncio.Semaphore(BURST_BUCKET)

async def safe_invoke(row: Dict[str, Any], chain) -> Tuple[Any, int]:
    """
    Return (response, remaining_requests_header)
    Remaining header == -1 if the header is missing (public OpenAI).
    """
    prompt_tok_est = row.get("_tok_prompt_est", 1_600)  # rough fallback

    async with req_limiter, tok_limiter, burst_sem:     # 3 gates
        resp = await chain.ainvoke(row)

    # Reconcile token bucket (give back surplus)
    try:
        used = resp.usage.total_tokens                   # OpenAI SDK style
    except Exception:
        used = prompt_tok_est
    surplus = max(prompt_tok_est - used, 0)
    if surplus:
        tok_limiter.release(surplus)

    # Read Azure header if present
    headers = getattr(resp, "response_metadata", {}).get("headers", {})
    remaining = int(headers.get("x-ratelimit-remaining-requests", -1))

    return resp, remaining

async def _run_row_with_retry(
    row_dict: Dict[str, Any],
    chain,
    *,
    max_retries: int,
    initial_delay: float,
):
    for attempt in range(1, max_retries + 1):
        try:
            return await safe_invoke(row_dict, chain)
        except Exception as e:
            if attempt >= max_retries:
                raise RuntimeError(
                    f"Row {row_dict.get('index')} failed after {max_retries} attempts: {e}"
                ) from e
            await asyncio.sleep(initial_delay * 2 ** (attempt - 1))

async def process_df_rows(
    df: pd.DataFrame,
    chain,
    *,
    partial_every: int = 5_000,
    partial_dir: str = "partials",
    final_path: str = "classification_results.csv",
    max_retries: int = 5,
    initial_delay: float = 3.0,
    cols_mapping: Dict[str, str] = {
                "source_name": "news_source",
                "title": "article_title",
                "segments": "article_body",
                "company_names": "companies",
                "company_codes": "company_codes",
                "modification_date": "modification_date",
            },
    cols2append_mapping: Dict[str, str] = {
        "company_codes": "original_company_codes",
        "companies": "original_companies",
    }
) -> pd.DataFrame:

    stamp   = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    out_dir = Path(f"{partial_dir}_{stamp}")
    out_dir.mkdir(parents=True, exist_ok=True)

    # dataframe → list[dict]
    rows: List[Dict[str, Any]] = (
        df.rename(columns=cols_mapping)
        .reset_index()
        .to_dict(orient="records")
    )
    
    logging.debug(f"Processing {len(rows)} rows")

    results, failures = [], []
    pbar = tqdm(total=len(rows), desc="Rows processed", unit="row")

    # internal queue of scheduled tasks
    pending: List[asyncio.Task] = []

    async def spawn(row_dict):
        t = asyncio.create_task(
            _run_row_with_retry(
                row_dict, chain,
                max_retries=max_retries,
                initial_delay=initial_delay,
            )
        )
        t.row_meta = row_dict
        pending.append(t)

    # prime pump with burst_bucket tasks
    for row in rows[:BURST_BUCKET]:
        await spawn(row)
    row_iter = iter(rows[BURST_BUCKET:])
    logging.debug(f"Primed with {len(pending)} tasks")
    
    while pending:
        # wait for the next task to finish
        done, _ = await asyncio.wait(pending, return_when=asyncio.FIRST_COMPLETED)
        for task in done:
            logging.debug(f"Task {task} done")
            pending.remove(task)
            try:
                resp, remaining = task.result()
                logging.debug(f"Task {task} result: {resp}")
                rec = resp.model_dump() if hasattr(resp, "model_dump") else resp
                for col, new_col in cols2append_mapping.items():
                    rec[new_col] = task.row_meta.get(col, "")
                # rec["original_company_codes"] = task.row_meta.get("company_codes", "")
                # rec["original_companies"]     = task.row_meta.get("companies", "")
                rec["index"]                  = task.row_meta.get("index") # always present
                results.append(rec)
            except Exception:
                logging.error(f"Task {task} failed: {task.exception()}")
                failures.append(task.row_meta.get("index"))

            pbar.update(1)

            # ---- adaptive part: widen/narrow the burst semaphore ----
            if remaining >= 0:
                # we finished one, so one slot is free; add any EXTRA free slots
                extra = max(remaining - burst_sem._value, 0)
                for _ in range(extra):
                    burst_sem.release()

            # periodic checkpoint
            if len(results) and len(results) % partial_every == 0:
                pd.DataFrame(results).to_csv(
                    out_dir / f"partial_{len(results)}.csv", index=False
                )
                print(f"[checkpoint] {len(results)} rows → {out_dir}")

            # schedule next row if we have more data
            try:
                next_row = next(row_iter)
                await spawn(next_row)
            except StopIteration:
                pass   # no more rows to enqueue

    pbar.close()

    logging.info(f"Processing complete: {len(results)} rows processed")
    
    # final save
    df_final  = pd.DataFrame(results)
    final_csv = Path(f"{final_path}_{stamp}.csv")
    final_csv.parent.mkdir(parents=True, exist_ok=True)
    df_final.to_csv(final_csv, index=False)
    print(f"[done] {len(results)} rows saved → {final_csv}")

    if failures:
        print(f"⚠️  {len(failures)} rows failed: {failures[:10]} …")

    return df_final


def run_pipeline_sync(*args, **kwargs):
    return asyncio.run(process_df_rows(*args, **kwargs))

async def run_pipeline_async(*args, **kwargs):
    return await process_df_rows(*args, **kwargs)

In [5]:
# df_results = await run_pipeline_async(
#     df,
#     so_chain,
#     partial_dir=f"../../extractions/{job_id}/partial_outputs",
#     final_path=f"../../extractions/{job_id}/co_classification_results",
#     max_retries=4,
#     initial_delay=3.0,
# ) # 4 hours / 135k entries

## Merging and filtering

Notebook died after VS code page reloading :man-shrugging:

In [3]:
df_results = pd.read_csv(f"../../extractions/{job_id}/co_classification_results_2025-05-21_14-48-22.csv")
df_results

,is_co,target_company_code,subsidiary_company_code,is_relevant,co_stage,short_reasoning,original_company_codes,original_companies,index
0,False,NaN,NaN,False,Early,The article provides no substantive informatio...,"bafinz,banqnp,eqstry,ifansg,obbncw,otvers",Bundesanstalt für Finanzdienstleistungsaufsich...,11
1,False,NaN,NaN,False,Early,The article focuses on Cleanova's presence enh...,"alvmar,cmbndi,dbvpl,percoi","Alvarez & Marsal Inc,Aon PLC,Debevoise & Plimp...",0
2,False,NaN,NaN,False,Early,The article only provides disclosure instructi...,"frfxp,fsautt,lsexch,ttkovp","Equals Group PLC,Financial Conduct Authority,L...",9
3,False,NaN,NaN,False,Early,The article contains only regulatory disclosur...,"fsautt,lsexch,po,ttkovp","Financial Conduct Authority,London Stock Excha...",16
4,False,NaN,NaN,False,Early,The article does not provide any information a...,"cadwit,dlnred,euecar,euruno,fdic,irevs,mrvtec,...","Cadwalader, Wickersham & Taft LLP,UBS Investme...",17
...,...,...,...,...,...,...,...,...,...
135280,False,NaN,NaN,False,Early,The article discusses Bet365's potential exit ...,"btsfgl,cnyc,cvccap,drftkn,gamvc,grdmp,habsaa,p...","bet365 Group Limited,JPMorgan Chase & Co.,CVC ...",135281
135281,False,NaN,NaN,False,Early,The article concerns HMRC issuing stop notices...,hmrcu,HM Revenue and Customs,135273
135282,False,NaN,NaN,False,Early,The article discusses the founder of a betting...,"btsfgl,cnyc,cvccap,drftkn,gamvc,grdmp,habsaa,p...","bet365 Group Limited,JPMorgan Chase & Co.,CVC ...",135278
135283,False,NaN,NaN,False,Early,The article discusses potential sale considera...,"btsfgl,cnyc,cvccap,drftkn,gamvc,grdmp,habsaa,p...","bet365 Group Limited,JPMorgan Chase & Co.,CVC ...",135282


In [4]:
results_df = df_results.copy() #to torture it as much as I want
results_df=results_df.set_index('index').sort_index()
results_df['target_company_code'] = results_df['target_company_code'].fillna('')
results_df['subsidiary_company_code'] = results_df['subsidiary_company_code'].fillna('')
results_df

,is_co,target_company_code,subsidiary_company_code,is_relevant,co_stage,short_reasoning,original_company_codes,original_companies
index,,,,,,,,
0,False,,,False,Early,The article focuses on Cleanova's presence enh...,"alvmar,cmbndi,dbvpl,percoi","Alvarez & Marsal Inc,Aon PLC,Debevoise & Plimp..."
1,False,,,False,Early,The article discusses Cleanova's acquisition o...,"alvmar,cmbndi,dbvpl,percoi","Alvarez & Marsal Inc,Aon PLC,Debevoise & Plimp..."
2,False,,,False,Early,The article discusses an acquisition by Cleano...,"alvmar,cmbndi,dbvpl,percoi","Alvarez & Marsal Inc,Aon PLC,Debevoise & Plimp..."
3,False,,,False,Early,The article discusses a transition process and...,"alvmar,cmbndi,dbvpl,percoi","Alvarez & Marsal Inc,Aon PLC,Debevoise & Plimp..."
4,True,smwh,smwh,False,Early,WHSmith (smwh) is exploring strategic options ...,"bhchcm,brraic,bsbro,lsexch,smwh","British Chambers of Commerce,British Retail Co..."
...,...,...,...,...,...,...,...,...
135280,False,,,False,Early,The article discusses Bet365 exiting remote ga...,"btsfgl,cnyc,cvccap,drftkn,gamvc,grdmp,habsaa,p...","bet365 Group Limited,JPMorgan Chase & Co.,CVC ..."
135281,False,,,False,Early,The article discusses Bet365's potential exit ...,"btsfgl,cnyc,cvccap,drftkn,gamvc,grdmp,habsaa,p...","bet365 Group Limited,JPMorgan Chase & Co.,CVC ..."
135282,False,,,False,Early,The article discusses potential sale considera...,"btsfgl,cnyc,cvccap,drftkn,gamvc,grdmp,habsaa,p...","bet365 Group Limited,JPMorgan Chase & Co.,CVC ..."


In [5]:
merged_df = pd.merge(df, results_df, left_index=True, right_index=True, how='left', suffixes=['_original', '_new'])
merged_df

,source_name,title,company_codes,modification_date,region_codes,segments,num_segments,an,tile_id,company_names,is_co,target_company_code,subsidiary_company_code,is_relevant,co_stage,short_reasoning,original_company_codes,original_companies
0,Gulf Oil & Gas,Cleanova expands its product offering with the...,"alvmar,cmbndi,dbvpl,percoi",2025-01-10 07:13:57.825,"eurz,uk,weurz",Enhances Cleanova’s presence in the UK and bro...,4,GUONGA0020250109el170007b,1910620,"Alvarez & Marsal Inc,Aon PLC,Debevoise & Plimp...",False,,,False,Early,The article focuses on Cleanova's presence enh...,"alvmar,cmbndi,dbvpl,percoi","Alvarez & Marsal Inc,Aon PLC,Debevoise & Plimp..."
1,Gulf Oil & Gas,Cleanova expands its product offering with the...,"alvmar,cmbndi,dbvpl,percoi",2025-01-10 07:13:57.825,"eurz,uk,weurz","\n\nCleanova, a clean tech, market-leading glo...",4,GUONGA0020250109el170007b,1910621,"Alvarez & Marsal Inc,Aon PLC,Debevoise & Plimp...",False,,,False,Early,The article discusses Cleanova's acquisition o...,"alvmar,cmbndi,dbvpl,percoi","Alvarez & Marsal Inc,Aon PLC,Debevoise & Plimp..."
2,Gulf Oil & Gas,Cleanova expands its product offering with the...,"alvmar,cmbndi,dbvpl,percoi",2025-01-10 07:13:57.825,"eurz,uk,weurz","\n\nCleanova’s CEO, Javaid Riaz, commented, “T...",4,GUONGA0020250109el170007b,1910622,"Alvarez & Marsal Inc,Aon PLC,Debevoise & Plimp...",False,,,False,Early,The article discusses an acquisition by Cleano...,"alvmar,cmbndi,dbvpl,percoi","Alvarez & Marsal Inc,Aon PLC,Debevoise & Plimp..."
3,Gulf Oil & Gas,Cleanova expands its product offering with the...,"alvmar,cmbndi,dbvpl,percoi",2025-01-10 07:13:57.825,"eurz,uk,weurz",\n\nJavaid Riaz explained what the transition ...,4,GUONGA0020250109el170007b,1910623,"Alvarez & Marsal Inc,Aon PLC,Debevoise & Plimp...",False,,,False,Early,The article discusses a transition process and...,"alvmar,cmbndi,dbvpl,percoi","Alvarez & Marsal Inc,Aon PLC,Debevoise & Plimp..."
4,thesun.co.uk,HIGH STREET GIANT WHSmith ‘in secret talks to ...,"bhchcm,brraic,bsbro,lsexch,smwh",2025-01-26 07:33:12.758,"eurz,uk,weurz",The retail group have been in negotiations wit...,7,THESUK0020250125el1p001jl,1910746,"British Chambers of Commerce,British Retail Co...",True,smwh,smwh,False,Early,WHSmith (smwh) is exploring strategic options ...,"bhchcm,brraic,bsbro,lsexch,smwh","British Chambers of Commerce,British Retail Co..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135280,thetimes.co.uk,"Who could buy Bet365? Bankers salivate, but a ...","btsfgl,cnyc,cvccap,drftkn,gamvc,grdmp,habsaa,p...",2025-05-04 07:19:19.010,"eurz,uk,weurz",\n\nJP Morgan was completely unaware that Coat...,7,TIMEUK0020250503el53000jh,2940030,"bet365 Group Limited,JPMorgan Chase & Co.,CVC ...",False,,,False,Early,The article discusses Bet365 exiting remote ga...,"btsfgl,cnyc,cvccap,drftkn,gamvc,grdmp,habsaa,p...","bet365 Group Limited,JPMorgan Chase & Co.,CVC ..."
135281,thetimes.co.uk,"Who could buy Bet365? Bankers salivate, but a ...","btsfgl,cnyc,cvccap,drftkn,gamvc,grdmp,habsaa,p...",2025-05-04 07:19:19.010,"eurz,uk,weurz",\n\nBet365 has historically had a sizeable rem...,7,TIMEUK0020250503el53000jh,2940031,"bet365 Group Limited,JPMorgan Chase & Co.,CVC ...",False,,,False,Early,The article discusses Bet365's potential exit ...,"btsfgl,cnyc,cvccap,drftkn,gamvc,grdmp,habsaa,p...","bet365 Group Limited,JPMorgan Chase & Co.,CVC ..."
135282,thetimes.co.uk,"Who could buy Bet365? Bankers salivate, but a ...","btsfgl,cnyc,cvccap,drftkn,gamvc,grdmp,habsaa,p...",2025-05-04 07:19:19.010,"eurz,uk,weurz",\n\nBut a sale risks bringing Coates into conf...,7,TIMEUK0020250503el53000jh,2940032,"bet365 Group Limited,JPMorgan Chase & Co.,CVC ...",False,,,False,Early,The article discusses potential sale considera...,"btsfgl,cnyc,cvccap,drftkn,gamvc,grdmp,habsaa,p...","bet365 Group Limited,JPMorgan Chase & Co.,CVC ..."
135283,thetimes.co.uk,"Who could buy Bet365? Bankers salivate, but a ...","btsfgl,cnyc,cvccap,drftkn,gamvc

In [6]:
co_df = merged_df[merged_df.is_co & merged_df.is_relevant].copy()
co_df

,source_name,title,company_codes,modification_date,region_codes,segments,num_segments,an,tile_id,company_names,is_co,target_company_code,subsidiary_company_code,is_relevant,co_stage,short_reasoning,original_company_codes,original_companies
79,Investors Chronicle - Magazine and Web Content,Why the end of De La Rue is the end of an era,"cryamb,dlar,pskdzt,schr",2025-01-17 07:25:09.485,"eurz,uk,weurz","\n\nMeanwhile, Crystal Amber’s shareholders ha...",6,INVR000020250116el1g0002t,1912118,"Crystal Amber Fund Limited,De La Rue PLC,Crane...",True,dlar,pskdzt,True,Late,De La Rue is currently engaged in selling off ...,"cryamb,dlar,pskdzt,schr","Crystal Amber Fund Limited,De La Rue PLC,Crane..."
86,Reuters News,UPDATE 1-Getir founder vows to sue over Mubada...,mbdlic,2025-01-16 07:19:46.849,"asiaz,balkz,devgcoz,dvpcoz,eurz,gulfstz,meastz...","\n\nIn September, Turkey's competition board s...",5,LBA0000020250115el1f01blx,1912168,Mubadala Investment Company PJSC,True,mbdlic,getir,True,Early,Mubadala is acquiring Getir's profitable local...,mbdlic,Mubadala Investment Company PJSC
124,Western Morning News,Sutton Harbour Group given new deadlines to pa...,"dfvndk,nwb,rbsct,suttoh",2025-01-24 07:22:24.769,"eland,eurz,uk,weurz","The company, which in December 2024 announced ...",3,WMN0000020250123el1n00012,1913466,"Sky Harbour Group Corp.,National Westminster B...",True,suttoh,,True,Early,Sutton Harbour Group PLC is undertaking a debt...,"dfvndk,nwb,rbsct,suttoh","Sky Harbour Group Corp.,National Westminster B..."
159,CE NoticiasFinancieras,"""My goal is not for Castlelake to sell, but to...","binter,foursh,gosic,tpgcml",2025-01-23 07:25:51.277,"africaz,balea,canry,eecz,eurz,medz,nafrz,spain...","\n\nQ: But it is for sale.\n\nA: Well, I don't...",31,NFINCE0020250122el1m003k1,1913645,"Bankinter SA,Four Seasons Hotels Limited,GIC P...",True,binter,,True,Early,The company (Bankinter SA) is discussing asset...,"binter,foursh,gosic,tpgcml","Bankinter SA,Four Seasons Hotels Limited,GIC P..."
177,CE NoticiasFinancieras,"""My goal is not for Castlelake to sell, but to...","binter,foursh,gosic,tpgcml",2025-01-23 07:25:51.277,"africaz,balea,canry,eecz,eurz,medz,nafrz,spain...","\n\nBorja Escalada (Madrid, 1979) took the rei...",31,NFINCE0020250122el1m003k1,1913663,"Bankinter SA,Four Seasons Hotels Limited,GIC P...",True,gosic,,True,Early,The new CEO of Millenium Hospitality (gosic) h...,"binter,foursh,gosic,tpgcml","Bankinter SA,Four Seasons Hotels Limited,GIC P..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134818,GlobalData Company Profiles,Comcast Corp - History,"akmt,amce,amdcs,amronl,amulci,amzcom,applc,apu...",2025-05-16 07:24:23.764,"atlnt,eurz,namz,uk,usa,usga,uss,weurz",\n\nDate : 2023\n\nNew Products/Services : In ...,62,GLOMCP0020250515el5f00etv,2936780,"Akamai Technologies, Inc.,AMC Entertainment Ho...",True,comcst,hulul,True,Early,Comcast Corporation (comcst) announced plans t...,"akmt,amce,amdcs,amronl,amulci,amzcom,applc,apu...","Akamai Technologies, Inc.,AMC Entertainment Ho..."
134890,Public Companies News and Documents via PUBT,Canal+ SA - 2024 Annual Report (CANAL Annual R...,"amronl,dmot,dsnyw,fratv,gneau,hisgcl,lgelec,ls...",2025-05-08 07:35:21.098,"devgcoz,eecz,eeurz,eurz,fra,medz,namz,pol,uk,u...",\n\nperformers and are already contributing po...,45,LCDVP00020250507el5701ee2,2936872,"Warner Bros. Discovery, Inc.,Dailymotion SA,Th...",True,gneau,,True,Early,Vivendi SA is implementing significant cost-cu...,"amronl,dmot,dsnyw,fratv,gneau,hisgcl,lgelec,ls...","Warner Bros. Discovery, Inc.,Dailymotion SA,Th..."
135001,The Irish Times,PTSB chief praises takeover target Finance Ire...,"dsbnk,ncnbco,wesah",2025-05-11 07:24:43.401,"eecz,eurz,ire,weurz",\n\nFinance Ireland reported a day earlier tha...,6,IRTI000020250510el5a0001k,2937352,"permanent tsb,Bank of America Corporation,RBC ...",True,dsbnk,ncnbco,True,Early,"Finance Ireland, majority-owned by Pimco and p...","dsbnk,ncnbco,wesah","permanent tsb,B

In [7]:
cols2drop = ["original_company_codes", "original_companies", ]
cols2rename = {"segments": "article_fragment", "is_co": "is_carve_out", "co_stage": "carve_out_stage", 
            "short_reasoning": "reasoning", "modification_date":"date"}

co_df = co_df.drop(columns=cols2drop)
co_df = co_df.rename(columns=cols2rename)

co_df

,source_name,title,company_codes,date,region_codes,article_fragment,num_segments,an,tile_id,company_names,is_carve_out,target_company_code,subsidiary_company_code,is_relevant,carve_out_stage,reasoning
79,Investors Chronicle - Magazine and Web Content,Why the end of De La Rue is the end of an era,"cryamb,dlar,pskdzt,schr",2025-01-17 07:25:09.485,"eurz,uk,weurz","\n\nMeanwhile, Crystal Amber’s shareholders ha...",6,INVR000020250116el1g0002t,1912118,"Crystal Amber Fund Limited,De La Rue PLC,Crane...",True,dlar,pskdzt,True,Late,De La Rue is currently engaged in selling off ...
86,Reuters News,UPDATE 1-Getir founder vows to sue over Mubada...,mbdlic,2025-01-16 07:19:46.849,"asiaz,balkz,devgcoz,dvpcoz,eurz,gulfstz,meastz...","\n\nIn September, Turkey's competition board s...",5,LBA0000020250115el1f01blx,1912168,Mubadala Investment Company PJSC,True,mbdlic,getir,True,Early,Mubadala is acquiring Getir's profitable local...
124,Western Morning News,Sutton Harbour Group given new deadlines to pa...,"dfvndk,nwb,rbsct,suttoh",2025-01-24 07:22:24.769,"eland,eurz,uk,weurz","The company, which in December 2024 announced ...",3,WMN0000020250123el1n00012,1913466,"Sky Harbour Group Corp.,National Westminster B...",True,suttoh,,True,Early,Sutton Harbour Group PLC is undertaking a debt...
159,CE NoticiasFinancieras,"""My goal is not for Castlelake to sell, but to...","binter,foursh,gosic,tpgcml",2025-01-23 07:25:51.277,"africaz,balea,canry,eecz,eurz,medz,nafrz,spain...","\n\nQ: But it is for sale.\n\nA: Well, I don't...",31,NFINCE0020250122el1m003k1,1913645,"Bankinter SA,Four Seasons Hotels Limited,GIC P...",True,binter,,True,Early,The company (Bankinter SA) is discussing asset...
177,CE NoticiasFinancieras,"""My goal is not for Castlelake to sell, but to...","binter,foursh,gosic,tpgcml",2025-01-23 07:25:51.277,"africaz,balea,canry,eecz,eurz,medz,nafrz,spain...","\n\nBorja Escalada (Madrid, 1979) took the rei...",31,NFINCE0020250122el1m003k1,1913663,"Bankinter SA,Four Seasons Hotels Limited,GIC P...",True,gosic,,True,Early,The new CEO of Millenium Hospitality (gosic) h...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134818,GlobalData Company Profiles,Comcast Corp - History,"akmt,amce,amdcs,amronl,amulci,amzcom,applc,apu...",2025-05-16 07:24:23.764,"atlnt,eurz,namz,uk,usa,usga,uss,weurz",\n\nDate : 2023\n\nNew Products/Services : In ...,62,GLOMCP0020250515el5f00etv,2936780,"Akamai Technologies, Inc.,AMC Entertainment Ho...",True,comcst,hulul,True,Early,Comcast Corporation (comcst) announced plans t...
134890,Public Companies News and Documents via PUBT,Canal+ SA - 2024 Annual Report (CANAL Annual R...,"amronl,dmot,dsnyw,fratv,gneau,hisgcl,lgelec,ls...",2025-05-08 07:35:21.098,"devgcoz,eecz,eeurz,eurz,fra,medz,namz,pol,uk,u...",\n\nperformers and are already contributing po...,45,LCDVP00020250507el5701ee2,2936872,"Warner Bros. Discovery, Inc.,Dailymotion SA,Th...",True,gneau,,True,Early,Vivendi SA is implementing significant cost-cu...
135001,The Irish Times,PTSB chief praises takeover target Finance Ire...,"dsbnk,ncnbco,wesah",2025-05-11 07:24:43.401,"eecz,eurz,ire,weurz",\n\nFinance Ireland reported a day earlier tha...,6,IRTI000020250510el5a0001k,2937352,"permanent tsb,Bank of America Corporation,RBC ...",True,dsbnk,ncnbco,True,Early,"Finance Ireland, majority-owned by Pimco and p..."
135022,Sharecast,Synthomer offloads inorganic chemistry busines...,"heqned,yulc",2025-05-07 07:30:32.306,"eurz,uk,weurz",The London-listed firm noted that William Blyt...,2,SHAREC0020250506el560008d,2937621,"H2 Equity Partners,Synthomer PLC",True,yulc,heqned,True,Early,"Synthomer PLC designated William Blythe, part ..."


In [8]:
co_df.to_csv(f"../../extractions/{job_id}/co_articles.csv", index=False)

### What is missing?|
1. Decode predicted company codes
2. Transform into summary output 

In [ ]:
import sys, os
from dotenv import load_dotenv

from tqdm import tqdm
tqdm.pandas()
sys.path.append('../../')
load_dotenv('../../.env')

user_key = os.getenv("FACTIVA_SNAPSHOTS_USER_KEY")

from src.factiva_api.taxonomy import init_client, get_company_code_mapping
from src.mapping.company_mapper import extract_company_names

client = init_client(user_key)
companies_mapping = get_company_code_mapping(client)

co_df['target_company'] = co_df['target_company_code'].progress_apply(lambda x: extract_company_names(x, companies_mapping))
co_df['subsidiary_company'] = co_df['subsidiary_company_code'].progress_apply(lambda x: extract_company_names(x, companies_mapping))


100%|██████████| 2869/2869 [00:00<00:00, 1270692.52it/s]


### Creating summary output + using web search to make the best guess
One big prompt to rule them may overwhelm o3-mini -> split it into two in 'summarization.ipynb'

In [ ]:
from pydantic import BaseModel, Field

class CarveOutSummary(BaseModel):
   target_company: str = Field(description="Name of the target company, same as in the provided example")

   group: str = Field(description="Name of the financial group")
   group_hq: str = Field(description="Base country code (two letters) of the financial group HQ, i.e. UK, US, FI, NL, etc.")
   vertical: str = Field(description="Sector of the financial group, i.e. Banking, Insurance, Data, etc.")
   
   potential_disposal: str = Field(description="Potential subdivision to be disposed of, i.e. UK business, Insurance Arm, IP business, etc - specific parts of the business")
   disposal_company_name: str = Field(description="Name of the company to be disposed of, i.e. Swedbank AB, Nordea Bank AB, etc.")
   disposal_nc_sector: Literal["Financial Services", "Technology & Payments", "Healthcare","Services & Industrial Tech", "Other"] = Field(description="Corresponding NC sector of potential disposal company.")
   
   rationale: str = Field(description="Rationale for the carve-out opportunity, i.e. divestment of non-core assets, strategic refocusing, etc., 1-2 sentences")   
   article_quote: str = Field(description="Quote from the article that supports the conclusion")   
   
   relevant: bool = Field(description="Is the article relevant to the conditions of business request? Most importantly, look at the region - potential disposals must be European (EEA).")
   interest_score: float = Field(description="Interest score of the article, 0-1. 0 means no interest, 1 means high interest. The earlier the carve-out opportunity, the higher the interest score.")

summary_prompt = """
You are an expert investment professional specializing in identifying potential corporate carve-out opportunities in the financial services sector.
Your task is to analyze news articles detected as potential carve-out opportunities and extract relevant information about the target company, the group it belongs to, and the potential disposal.

A very important part of your task is to correctly identify the target company and the group it belongs to, as well as the potential disposal.
The target company is the one that may divest a subsidiary
The disposal company is the one that may be divested
The group is the parent company of the target company.

When making a decision, think step by step, and correctly answer the questions.
Using available web search, populate the fields related to target company, group, and potential disposal.

# Instructions:
    Carefully analyze the provided information and extract the relevant details to fill in the CarveOutSummary model.

CRITICAL EVALUATION CRITERIA:
1. MUST IDENTIFY AS CARVE-OUT PROSPECT:
   - Company signals intention to divest specific business units or segments
   - Company announces strategic refocusing that implies non-core units could be sold
   - New strategic plans mentioning focus on specific segments (implying others might be divested)
   - Language about simplifying corporate structure or streamlining operations
   - References to non-core assets, underperforming units, or subscale operations
   - Management discussing 'evaluating options' for specific business segments
    
2. MUST NOT BE INCORRECTLY CLASSIFIED AS CARVE-OUT:
   - Simple stake sales without business unit separation 
   - Private equity firms exiting investments (not corporate carve-outs)
   - IPO plans without specific business unit divestiture
   - Already completed carve-outs (though mention these in reasoning as potentially interesting)
   - General M&A activity without specific divestiture signals
   - Opinion pieces or columns without factual business announcements
    
3. TIMING RELEVANCE:
   - FOCUS ON FUTURE OPPORTUNITIES: Companies currently considering or likely to consider divestitures
   - DE-PRIORITIZE: Already completed transactions (though note these in reasoning)
    
4. KEY SIGNALS OF POTENTIAL CARVE-OUTS:
   - New CEO appointments or management changes
   - Announcements of strategic reviews or new strategic plans
   - Explicit focus on 'core businesses' or 'key segments'
   - Financial pressure (debt issues, dividend concerns, performance challenges)
   - Regulatory challenges that might prompt divestiture
   - Simplification of corporate structure
   - Discontinuation of operations in certain areas
    
When selecting target_company_code:
- If multiple companies might be considering divestitures, select the primary/most likely one
- For corporate groups where a parent may divest subsidiaries, consider both parent and subsidiary codes
- If a company is refocusing on specific segments, this implies other segments could be divested
- Do not pick the wrong company - target company is the one that may divest a subsidiary, a subsidiary is the one that may be divested
- Subsidiary company must be located in Europe (EEA) as specified in the business request
    
Business request: {business_request}

Analyze provided article and carve-out prediction justification.
If you do not have enough information to answer the questions, please say so.

source: {source_name}
title: {title}
date: {date}
article body: {article_fragment}
mentioned_companies: {companies}
mentioned_company_codes: {company_codes}
target_company_code: {target_company_code}
subsidiary_company_code: {subsidiary_company_code}
carve_out_stage: {carve_out_stage}
carve_out_reasoning: {reasoning}
"""

In [28]:
from langchain_core.prompts import PromptTemplate
from llm_utils.factory import LLMChainFactory

# llm = construct_llm(provider="openai",model_name='o3-mini', temperature=0)
factory = LLMChainFactory(model_name='o3-mini', provider="openai")
summary_template = PromptTemplate.from_template(template=summary_prompt, partial_variables={'business_request': business_request})
runnable = factory.build_search_runnable_with_structured_output(pydantic_model=CarveOutSummary)
summary_chain = summary_template | runnable


In [31]:

summary_cols_mapping = {
    "company_names": "companies",
}

summary_df_results = await run_pipeline_async(
    co_df[:],
    summary_chain,
    partial_dir=f"../../extractions/{job_id}/partial_outputs_summary",
    final_path=f"../../extractions/{job_id}/co_summary_results",
    max_retries=4,
    initial_delay=3.0,
    partial_every=2800,
    cols_mapping=summary_cols_mapping,
    cols2append_mapping={'date':'date'}
)

Rows processed:   0%|          | 0/2869 [00:00<?, ?row/s]

Rows processed:  98%|█████████▊| 2799/2869 [10:14<00:15,  4.66row/s]

[checkpoint] 2800 rows → ../../extractions/dj-synhub-extraction-lkbi9fy6zepu8rcjuxqhjwkbld52wgt0-ouutovygqc/partial_outputs_summary_2025-05-21_22-23-50


Rows processed: 100%|██████████| 2869/2869 [20:36<00:00,  2.32row/s] 
INFO:root:Processing complete: 2869 rows processed


[done] 2869 rows saved → ../../extractions/dj-synhub-extraction-lkbi9fy6zepu8rcjuxqhjwkbld52wgt0-ouutovygqc/co_summary_results_2025-05-21_22-23-50.csv


In [32]:
summary_df = summary_df_results.copy() #to torture it
summary_df = summary_df[~summary_df.irrelevant]
summary_df = summary_df.set_index('index').sort_index()
summary_df

,target_company,group,group_hq,vertical,potential_disposal,disposal_company_name,rationale,article_quote,irrelevant,interest_score,date
index,,,,,,,,,,,
79,De La Rue PLC,De La Rue PLC,UK,Security Printing,Authentication Division and potentially other ...,De La Rue Authentication Division,De La Rue is under financial pressure and is s...,"Last July, the IC's Simon Thompson spotted tha...",False,0.8,2025-01-17 07:25:09.485
86,Getir,Mubadala Investment Company PJSC,AE,Investment,Non-core businesses of Getir,Getir,Mubadala plans to separate non-core businesses...,"In exchange, it would separate noncore busines...",False,0.8,2025-01-16 07:19:46.849
124,Sutton Harbour Group PLC,Sutton Harbour Group PLC,UK,Real Estate,Asset sales,Sutton Harbour Group PLC,Sutton Harbour Group is undertaking a debt red...,"The company, which in December 2024 announced ...",False,1.0,2025-01-24 07:22:24.769
159,Bankinter SA,Bankinter Group,ES,Banking,Specific Asset (details not specified),Not specified,Bankinter SA is focusing on asset rotation and...,In the strategic plan it appears as an asset t...,False,0.8,2025-01-23 07:25:51.277
177,Millenium Hospitality,Millenium Hospitality Group,Spain,Hospitality,European market operations,Millenium Hospitality (European operations),The new CEO has announced a strategic plan to ...,"The executive has landed in the socimi, along ...",False,1.0,2025-01-23 07:25:51.277
...,...,...,...,...,...,...,...,...,...,...,...
134681,Banco Santander S.A.,Banco Santander S.A.,ES,Banking,Polish asset management business,Santander Bank Polska S.A.,Banco Santander is selling a 49% stake in Sant...,Santander announces the sale of 49% of Santand...,False,1.0,2025-05-08 07:33:18.578
134890,Vivendi SA,Vivendi SA,FR,Media and Entertainment,PayTV Channels,Canal+ SA,Vivendi SA is focusing on cost-cutting and ope...,We undertook a wide range of cost-cutting meas...,False,0.8,2025-05-08 07:35:21.098
135001,Finance Ireland,Pimco,US,Investment Management,Mortgage Lending Business,Finance Ireland,Finance Ireland decided to exit mortgage lendi...,The bank decided in March to get out of mortga...,False,0.8,2025-05-11 07:24:43.401


In [34]:
summary_df.target_company.value_counts()

target_company
UBS Group AG            68
Galapagos NV            38
HSBC Holdings PLC       34
NatWest Group PLC       32
Telefónica SA           31
                        ..
Forafric Global PLC      1
Lindex Group Oyj         1
Mallinckrodt plc         1
Leonardo S.p.A           1
Bet365 Group Limited     1
Name: count, Length: 986, dtype: int64

In [57]:
def most_common(series):
    return series.value_counts().index[0] if not series.empty else None

# Function to concatenate unique values with optional limit
def set_concat(series, max_items=None, max_chars=None):
    """
    Concatenate unique values with limits on items and characters
    """
    unique_values = pd.Series(series).dropna().astype(str).unique()
    
    # Apply item limit if specified
    if max_items and len(unique_values) > max_items:
        truncated = unique_values[:max_items]
        result = " | ".join(truncated) + f" (+ {len(unique_values) - max_items} more)"
    else:
        result = " | ".join(unique_values) if len(unique_values) > 0 else ""
    
    # Apply character limit if specified
    if max_chars and len(result) > max_chars:
        result = result[:max_chars] + "..."
        
    return result
full_df = co_df[['article_fragment','carve_out_stage']].merge(summary_df, left_index=True, right_index=True, how='left', suffixes=['_original', '_new'])
# Group by target_company with different aggregations for each column
grouped_summary = full_df.groupby("target_company").agg({
    "group": most_common,  # Most popular group
    "group_hq": most_common,  # Most common HQ

    "vertical": most_common,  # Most common vertical
    "potential_disposal": lambda x: set_concat(x, 3),  # Concatenate unique values, limit to 3
    "disposal_company_name": lambda x: set_concat(x, 3),  # Concatenate unique values, limit to 3

    "rationale": lambda x: set_concat(x, 3),  # Concatenate unique rationales, limit to 2
    "date": "min",
    "interest_score": "mean",  # Average interest score

    "carve_out_stage": most_common,  # Most common carve out stage

    "article_quote": lambda x: set_concat(x, 3),  # Concatenate unique quotes, limit to 3    
    "article_fragment": lambda x: set_concat(x, max_items=2, max_chars=500),  # Limit chars for article fragments
    
})

grouped_summary['interest_score'] = grouped_summary['interest_score'].round(2)
grouped_summary['date'] = pd.to_datetime(grouped_summary['date']).dt.strftime('%Y-%m-%d')
grouped_summary.reset_index(inplace=True)

### Save to nice excel

In [ ]:
from src.utils.excel_export import export_summary_to_excel
export_file = export_summary_to_excel(grouped_summary)
print(f"Exported to {export_file}")

Exported to ../../extractions/dj-synhub-extraction-lkbi9fy6zepu8rcjuxqhjwkbld52wgt0-ouutovygqc/carveouts_summary_grouped.xlsx
